## 60-degree roto-translation mechanism of <110>-oriented dumbbells in BCC lattice.

In this notebook, we'll create and save a onsager transport coefficient calculator for the 60-degree roto-translation mechanism of <110> dumbbells in a BCC lattice, as per the data in the paper by Messina et. al. at https://doi.org/10.1016/j.actamat.2020.03.038

To do this, we need to create a 1nn3 thermodynamic shell (all sites up to the first of first of first nearest neighbors of the solute), large enough to encompass all the solute-dumbbell configurations mentioned in this paper, and also extract out the necessary 60-degree re-orienting dumbbell jumps from those created created by our code.

In [1]:
import numpy as np
from onsager.crystal import Crystal
from onsager.OnsagerCalc import dumbbellMediated
from onsager.crystal import DB_disp, DB_disp4, pureDBContainer, mixedDBContainer
from onsager.DB_structs import dumbbell, SdPair, jump, connector

In [2]:
# we'll start by making a BCC lattice with the lattice parameter of iron.
# Then we'll modify the jumpnetworks to keep only the 60 degree reorientational jumps.
latt = np.array([[1., 0., 0.], [0., 1., 0.], [0., 0., 1.]]) * 0.2831
Fe = Crystal(latt, [[np.array([0., 0., 0.]), np.array([0.5, 0.5, 0.5])]], ["Fe"])

# Now give it the orientations - for BCC it's [110] - make the dumbbell length the same as atomic
# diameter of iron (0.126 * 2). Note that this is only a nominal length. Jump Distances are only
# considered site-to-site as allowed by the invariance principle of mass transport.
o = np.array([1.,1.,0.])/np.linalg.norm(np.array([1.,1.,0.]))*0.126*2
famp0 = [o.copy()]
family = [famp0]
pdbcontainer_fe = pureDBContainer(Fe, 0, family)
mdbcontainer_fe = mixedDBContainer(Fe, 0, family)

# Next, we generate the pure and mixed dumbbell jump networks.
# We set the cutoff jump distance to a little more than the corner-to-body center distance
# jset0 contains pure dumbbell jumps, and jset2 contains mixed dumbbell jumps
# We keep collision thresholds to a small value of 0.01 nm, so that
# as many jumps as possible are initially built.
# We'll extract the jumps we need from those.
jset0, jset2 = pdbcontainer_fe.jumpnetwork(0.25, 0.01, 0.01), mdbcontainer_fe.jumpnetwork(0.25, 0.01, 0.01)
print(Fe)

#Lattice:
  a1 = [0.14155 0.14155 0.14155]
  a2 = [-0.14155  0.14155 -0.14155]
  a3 = [-0.14155 -0.14155  0.14155]
#Basis:
  (Fe) 0.0 = [0. 0. 0.]


In [3]:
# Modify jnet0
# 60-degree reorienting jumps are the jumps with the shortest total displacement
# of atoms involved. So we'll sort the jumps by the total displacements of
# the 3-atom (two atoms of the initial dumbbell and the atom at the final site)

jnet0 = jset0[0] # Contains jump objects
jnet0_indexed = jset0[1] # Contains jumps with indices of the dumbbells in the container objects.

# Let's try to sort the jumps according to closest distance
# except rotational jumps, we don't want them.

def sortkey(entry):
    jmp = jnet0[entry][0]
    or1 = pdbcontainer_fe.iorlist[jmp.state1.iorind][1]
    or2 = pdbcontainer_fe.iorlist[jmp.state2.iorind][1]
    dx = DB_disp(pdbcontainer_fe, jmp.state1, jmp.state2)
    dx1 = np.linalg.norm(jmp.c1*or1/2.)
    dx2 = np.linalg.norm(dx + jmp.c2*or2/2. - jmp.c1*or1/2.)
    dx3 = np.linalg.norm(-jmp.c2*or2/2.)
    return dx1+dx2+dx3

z = np.zeros(3)
indices = []
for jt, jlist in enumerate(jnet0):
    if np.allclose(jnet0_indexed[jt][0][1], z):
        continue
    indices.append(jt)

ind_sort = sorted(indices, key=sortkey)[:1]
len(ind_sort)

1

In [4]:
# Let's print the jump network to see if we have the right jumps

# First, we print out the dumbbell orientations
print("Pure dumbbell (i, or) list:")
for tup in pdbcontainer_fe.iorlist:
    print(tup)
    
print()

# For a pure dumbbell, the (i, or) indices (iorInd) correspond to (basis site, orientation vector) pairs.
# The corresponding values can be found in the "iorlist" we just printed out.

# c1 = 1 means the atom at the head of the initial orientation vector jumps.
# c1 = -1 means the atom at the tail of the initial orientation vector jumps.

# c2 = 1 means the atom lands at the head of the final orientation vector.
# c2 = -1 means the atom lands at the tail of the final orientation vector.

j = jnet0[ind_sort[0]][0]
print(j)
print("dumbbell positions and orientiations in Cartesian coordinates:")
print("initial dumbbell position and orientiation: ", np.dot(Fe.lattice, j.state1.R),", ",
      pdbcontainer_fe.iorlist[j.state1.iorind][1])
print("Final dumbbell position and orientiation: ", np.dot(Fe.lattice, j.state2.R), ", ",
     pdbcontainer_fe.iorlist[j.state2.iorind][1])

Pure dumbbell (i, or) list:
(0, array([-0.17819091, -0.17819091,  0.        ]))
(0, array([ 0.        ,  0.17819091, -0.17819091]))
(0, array([-0.17819091,  0.        , -0.17819091]))
(0, array([ 0.17819091,  0.        , -0.17819091]))
(0, array([ 0.17819091, -0.17819091,  0.        ]))
(0, array([ 0.        , -0.17819091, -0.17819091]))

Jump object:
Initial state:
	dumbbell : (i, or) index = 0, lattice vector = [0 0 0]
Final state:
	dumbbell : (i, or) index = 2, lattice vector = [-1  0  0]
Jumping from c1 = 1 to c2 = -1

dumbbell positions and orientiations in Cartesian coordinates:
initial dumbbell position and orientiation:  [0. 0. 0.] ,  [-0.17819091 -0.17819091  0.        ]
Final dumbbell position and orientiation:  [-0.14155 -0.14155 -0.14155] ,  [-0.17819091  0.         -0.17819091]


In [5]:
# let's check the re-orientiation angle of the jump
cos = np.dot(pdbcontainer_fe.iorlist[j.state1.iorind][1], pdbcontainer_fe.iorlist[j.state2.iorind][1])
cos /= np.linalg.norm(pdbcontainer_fe.iorlist[j.state1.iorind][1])
cos /= np.linalg.norm(pdbcontainer_fe.iorlist[j.state2.iorind][1])
np.arccos(cos * (-j.c1 * j.c2)) * 180 / np.pi

60.00000000000001

In [6]:
# take the jumps we want.
jset0new = ([jnet0[i] for i in ind_sort], [jnet0_indexed[i] for i in ind_sort])

In [7]:
# Now, we modify the mixed dumbbell jumpnetwork.
# We'll include all the jumps there are in the paper
# They are: First and second lowest transition jumps, 60 deg on-site rotation, 90-deg onsite rotation.
jnet2 = jset2[0]
jnet2_indexed = jset2[1]
# Let's try to sort the jumps according to closest distance
# we don't want any rotational jumps.

def sortkey2(entry):
    jmp = jnet2[entry][0]
    or1 = mdbcontainer_fe.iorlist[jmp.state1.db.iorind][1]
    or2 = mdbcontainer_fe.iorlist[jmp.state2.db.iorind][1]
    dx = DB_disp(mdbcontainer_fe, jmp.state1, jmp.state2)
    # c1 and c2 are always +1 for mixed dumbbell jumps.
    dx1 = np.linalg.norm(jmp.c1*or1/2.)
    dx2 = np.linalg.norm(dx + jmp.c2*or2/2. - jmp.c1*or1/2.)
    dx3 = np.linalg.norm(-jmp.c2*or2/2.)
    return dx1+dx2+dx3

z = np.zeros(3)
indices2 = []
indices_rot = []
for jt, jlist in enumerate(jnet2):
    if np.allclose(jnet2_indexed[jt][0][1], z):
        jmp = jlist[0]

        or1 = mdbcontainer_fe.iorlist[jmp.state1.db.iorind][1]
        or2 = mdbcontainer_fe.iorlist[jmp.state2.db.iorind][1]
        
        if np.allclose(np.dot(or1,or2)/(np.linalg.norm(or1)*np.linalg.norm(or2)), np.cos(np.pi/3.)):
#             print("got 60 deg jump at {}".format(jt))
            indices_rot.append(jt)
            continue
        elif np.allclose(np.dot(or1,or2), 0.):
#             print("got 90 deg jump at {}".format(jt))
            indices_rot.append(jt)
            continue
        else:
            continue
    indices2.append(jt)
ind_sort2 = sorted(indices2, key=sortkey2)[:2]
indices2all = ind_sort2[:1] # get the jump with the smallest total displacement
print(indices2all)

[14]


In [8]:
# Now let's check if we've got the correct mixed dumbbell jumps.
# For mixed dumbbells, the solute atom is always at the head of the dumbbells,
# So there is no need to look as c1 and c2.

j = jnet2[ind_sort2[0]][0]

print("dumbbell positions and orientiations in Cartesian coordinates:")
print("initial dumbbell position and orientiation: ", np.dot(Fe.lattice, j.state1.db.R),", ",
      mdbcontainer_fe.iorlist[j.state1.db.iorind][1])
print("Final dumbbell position and orientiation: ", np.dot(Fe.lattice, j.state2.db.R), ", ",
     mdbcontainer_fe.iorlist[j.state2.db.iorind][1])

dumbbell positions and orientiations in Cartesian coordinates:
initial dumbbell position and orientiation:  [0. 0. 0.] ,  [ 0.          0.17819091 -0.17819091]
Final dumbbell position and orientiation:  [-0.14155  0.14155 -0.14155] ,  [ 0.17819091 -0.17819091  0.        ]


In [9]:
# let's check the re-orientiation angle of the jump
v1 = mdbcontainer_fe.iorlist[j.state1.db.iorind][1] # the initial orientation vector
v2 = mdbcontainer_fe.iorlist[j.state2.db.iorind][1]
cos = np.dot(v1, v2)
cos /= np.linalg.norm(v1)
cos /= np.linalg.norm(v2)
np.arccos(-cos) * 180 / np.pi

60.00000000000001

In [10]:
jset2new = ([jnet2[i] for i in indices2all], [jnet2_indexed[i] for i in indices2all])

In [11]:
# Now initialize the onsager calculator.
# Now, we construct the Onsager calculator with the pure and mixed dumbbell jump networks.
# These will be used to build the omega1 jump networks as well.
# We also set the collision threshold from omega43 jumps to 0.01 nm initially, so
# that as many jumps as possible are built, from which we'll extract the necessary
# 60-degree roto-translation jumps. The cutoff distance for the omega43 jumps is also
# set to 0.26 nm, sligthly larger than the nearest neighbor distance in the BCC iron unit cell.
import time
start = time.time()
onsagercalculator = dumbbellMediated(pdbcontainer_fe, mdbcontainer_fe, jset0new, jset2new, 0.27,
                                     0.01, 0.01, 0.01, NGFmax=4, Nthermo=3)
print("onsager calculator initiation time = {}".format(time.time() - start))

initializing thermo
initializing kin
generating thermodynamic shell
built shell 1: time - 0.03377389907836914
built shell 2: time - 0.984337568283081
built shell 3: time - 4.814657688140869
grouped states by symmetry: 10.152561902999878
built mixed dumbbell stars: 0.0007784366607666016
built jtags2: 6.937980651855469e-05
built mixed indexed star: 0.009060144424438477
building star2symlist : 0.00021958351135253906
building bare, mixed index dicts : 0.00035119056701660156
thermodynamic shell generated: 25.945303201675415
Total number of states in Thermodynamic Shell - 546, 12
generating kinetic shell
built shell 1: time - 0.03314065933227539
built shell 2: time - 1.030273199081421
built shell 3: time - 4.7829673290252686
built shell 4: time - 12.203553915023804
grouped states by symmetry: 40.35102701187134
built mixed dumbbell stars: 0.0007579326629638672
built jtags2: 6.461143493652344e-05
built mixed indexed star: 0.008876323699951172
building star2symlist : 0.0003809928894042969
build

In [12]:
# The onsager calculator initially builds all possible omega43 jumps.
# So next, we extract our necessary 60-degree roto-translational omega4-omega3 jumps
jnet43 = onsagercalculator.jnet43
jnet43_indexed = onsagercalculator.jnet43_indexed
# Let's try to sort the jumps according to closest distance
# We assume that association and dissociation occurs via Johnson mechanism.

def sortkey3(entry):
    jmp = jnet43[entry][0] # This is an omega4 jump
    if not jmp.c2 == -1:
        print(c2)
    or1 = pdbcontainer_fe.iorlist[jmp.state1.db.iorind][1]
    or2 = mdbcontainer_fe.iorlist[jmp.state2.db.iorind][1]
    dx = DB_disp4(pdbcontainer_fe, mdbcontainer_fe, jmp.state1, jmp.state2)
    # remember that c2 is -1 for an omega4 jump
    dx1 = np.linalg.norm(jmp.c1*or1/2.)
    dx2 = np.linalg.norm(dx - or2/2. - jmp.c1*or1/2.)
    dx3 = np.linalg.norm(jmp.c2*or2/2.)
    return dx1+dx2+dx3

z = np.zeros(3)
indices43 = []
for jt, jlist in enumerate(jnet43):
    if np.allclose(jnet43_indexed[jt][0][1], z):
        continue
    indices43.append(jt)    
ind_sort43 = sorted(indices43, key=sortkey3)[:1]
print(ind_sort43)

[14]


In [14]:
# Let's take a look at the extracted omega 43 jumps.
# Since they have the same transition state, they are all grouped together in
# the same list. Even indices in this list (0, 2, 4...) will give us omega4 jumps.
# The odd indices (1, 3, 5,...) correspond to omega3 jumps.

# For example jnet43[some_list_index][0] is an omega4 jump, while jnet43[some_list_index][1] is an omega3 jump.

# First, we print out the dumbbell (basis site, orientation) list
print("Pure dumbbell (basis site, orientation) or (i, or) list:")
for tup in pdbcontainer_fe.iorlist:
    print(tup)
    
print()

# Then let's print out the mixed dumbbell (basis site, orientation) list
print("Mixed dumbbell (basis site, orientation) or (i, or) list:")
for tup in mdbcontainer_fe.iorlist:
    print(tup)
print()

# Remmber that in an omega4 jump, the self-interstitial jumps, and the solute is always considered
# to be at the head of the dumbbell, so c2 will always be -1 (since the self-interstitial is considered
# to land at the tail of the mixed dumbbell).

j = jnet43[ind_sort43[0]][0] # This is an omega4 jump.
print(j,"\n")
print("dumbbell positions and orientiations in Cartesian coordinates:")
print("initial dumbbell position and orientiation: ", np.dot(Fe.lattice, j.state1.db.R),", ",
      pdbcontainer_fe.iorlist[j.state1.db.iorind][1]) # the initial state is a pure dumbbell
print("Final dumbbell position and orientiation: ", np.dot(Fe.lattice, j.state2.db.R), ", ",
     mdbcontainer_fe.iorlist[j.state2.db.iorind][1])

Pure dumbbell (basis site, orientation) or (i, or) list:
(0, array([-0.17819091, -0.17819091,  0.        ]))
(0, array([ 0.        ,  0.17819091, -0.17819091]))
(0, array([-0.17819091,  0.        , -0.17819091]))
(0, array([ 0.17819091,  0.        , -0.17819091]))
(0, array([ 0.17819091, -0.17819091,  0.        ]))
(0, array([ 0.        , -0.17819091, -0.17819091]))

Mixed dumbbell (basis site, orientation) or (i, or) list:
(0, array([-0.17819091, -0.17819091,  0.        ]))
(0, array([ 0.        ,  0.17819091, -0.17819091]))
(0, array([-0.17819091,  0.        , -0.17819091]))
(0, array([ 0.17819091,  0.        , -0.17819091]))
(0, array([-0.17819091,  0.        ,  0.17819091]))
(0, array([ 0.17819091, -0.17819091,  0.        ]))
(0, array([0.17819091, 0.17819091, 0.        ]))
(0, array([ 0.        , -0.17819091,  0.17819091]))
(0, array([ 0.        , -0.17819091, -0.17819091]))
(0, array([0.        , 0.17819091, 0.17819091]))
(0, array([-0.17819091,  0.17819091,  0.        ]))
(0, ar

In [16]:
# let's check the re-orientiation angle of the jump
v1 = pdbcontainer_fe.iorlist[j.state1.db.iorind][1] # the initial orientation vector
v2 = mdbcontainer_fe.iorlist[j.state2.db.iorind][1]
cos = np.dot(v1, v2)
cos /= np.linalg.norm(v1)
cos /= np.linalg.norm(v2)
np.arccos(cos * (-j.c1 * j.c2)) * 180 / np.pi

60.00000000000001

In [17]:
# Now we regenerate the omega43 information in the calculator
# to only contain the jump type we need, stored in the list ind_sort43

onsagercalculator.regenerate43(ind_sort43)

In [18]:
# Then we save the onsager calculator so we can re-use it.
import pickle
with open('FeX_60RT.pkl','wb') as fl:
    pickle.dump(onsagercalculator,fl)